### Amazon reviews

Loading the dataset a printing some data about it

In [1]:
from quapy.data.reader import from_text
from quapy.data.base import LabelledCollection
import os

base_path = "/media/nas/pgonzalez/quant_datasetshift/datasets/reviews"
subsample_size = 10000 #Subsample for the sake of speed

domains_full = {
    "electronics": LabelledCollection.load(os.path.join(base_path, "Electronics.txt"), from_text),
    "books": LabelledCollection.load(os.path.join(base_path, "Books.txt"), from_text),
    "cds": LabelledCollection.load(os.path.join(base_path,  "CDs_and_Vinyl.txt"), from_text),
    "home": LabelledCollection.load(os.path.join(base_path, "Home_and_Kitchen.txt"), from_text)
}

loading /media/nas/pgonzalez/quant_datasetshift/datasets/reviews/Electronics.txt: 100%|██████████| 2083988/2083988 [00:02<00:00, 1005359.99it/s]
loading /media/nas/pgonzalez/quant_datasetshift/datasets/reviews/Books.txt:  92%|█████████▏| 7969282/8623133 [00:09<00:00, 897788.87it/s]

format error in Funnily enough, that makes it a better book.  I guess the author has come to realise that nine of his thirteen points were about money.  So he got a very funny (but impossibly lazy, he could not even be bothered to dig beyond his personal experience for new stories/examples) co-author to repackage those ideas and hey, presto, a better-focused book is on the shelves.  Also, a major beef of mine with psychologists is they have to get sex into everywhere, a fact that makes it impossible to recommend their books to my mom or even to read their work on the tube.  With point #6 expunged, this is indeed a book you can recommend to your mom and a book you can read in the tube with little risk of embarrassment.  Modulo the jarringly frequent invocation of a dominatrix throughout the text, that is.  So to all those of you whove never read Dan Ariely before, you can now skip his original opus.  Buy Small Change instead, supplement it with The Honest Truth about Dishonesty and be o

loading /media/nas/pgonzalez/quant_datasetshift/datasets/reviews/Books.txt: 100%|██████████| 8623133/8623133 [00:09<00:00, 877801.71it/s]
loading /media/nas/pgonzalez/quant_datasetshift/datasets/reviews/CDs_and_Vinyl.txt: 100%|██████████| 1140936/1140936 [00:01<00:00, 974228.82it/s]
loading /media/nas/pgonzalez/quant_datasetshift/datasets/reviews/Home_and_Kitchen.txt: 100%|██████████| 1879697/1879697 [00:02<00:00, 865411.58it/s] 


In [2]:
# Remove 3 starts reviews and binarize
for domain, lc in domains_full.items():
    instances, labels = lc.Xy
    # Remove 3 stars reviews
    labels, instances = labels[labels!=3], instances[labels!=3]
    # Binarize
    labels[labels<3] = 0
    labels[labels>3] = 1
    domains_full[domain] = LabelledCollection(instances,labels)

### Printing some data about each domain

In [9]:
for name, domain in domains_full.items():
    print("%s:" % name)
    print(domain.stats())

electronics:
#instances=1889965, type=<class 'str'>, #features=?, #classes=[0 1], prevs=[0.300, 0.700]
{'instances': 1889965, 'type': <class 'str'>, 'features': '?', 'classes': array([0, 1]), 'prevs': '[0.300, 0.700]'}
books:
#instances=7813813, type=<class 'str'>, #features=?, #classes=[0 1], prevs=[0.181, 0.819]
{'instances': 7813813, 'type': <class 'str'>, 'features': '?', 'classes': array([0, 1]), 'prevs': '[0.181, 0.819]'}
cds:
#instances=1052258, type=<class 'str'>, #features=?, #classes=[0 1], prevs=[0.113, 0.887]
{'instances': 1052258, 'type': <class 'str'>, 'features': '?', 'classes': array([0, 1]), 'prevs': '[0.113, 0.887]'}
home:
#instances=1695843, type=<class 'str'>, #features=?, #classes=[0 1], prevs=[0.292, 0.708]
{'instances': 1695843, 'type': <class 'str'>, 'features': '?', 'classes': array([0, 1]), 'prevs': '[0.292, 0.708]'}


### Computing how different is each domain
He idea is to train a simple classifier for each domain and apply that to the others

In [47]:
from quapy.data.base import Dataset

datasets = {}

#Create the datasets
for name, domain_full in domains_full.items():
    print("[%s] Getting a subsample of the domain" % name)
    domain = domain_full.sampling(subsample_size,0.5,0.5)
    print("[%s] Splitting in train and test" % name)
    train, test = domain.split_stratified(train_prop=0.6)
    datasets[name] = Dataset(train,test)
    datasets[name].stats(show=True)

[electronics] Getting a subsample of the domain
[electronics] Splitting in train and test
Dataset= #tr-instances=6000, #te-instances=4000, type=<class 'str'>, #features=?, #classes=[0 1], tr-prevs=[0.500, 0.500], te-prevs=[0.500, 0.500]
[books] Getting a subsample of the domain
[books] Splitting in train and test
Dataset= #tr-instances=6000, #te-instances=4000, type=<class 'str'>, #features=?, #classes=[0 1], tr-prevs=[0.500, 0.500], te-prevs=[0.500, 0.500]
[cds] Getting a subsample of the domain
[cds] Splitting in train and test
Dataset= #tr-instances=6000, #te-instances=4000, type=<class 'str'>, #features=?, #classes=[0 1], tr-prevs=[0.500, 0.500], te-prevs=[0.500, 0.500]
[home] Getting a subsample of the domain
[home] Splitting in train and test
Dataset= #tr-instances=6000, #te-instances=4000, type=<class 'str'>, #features=?, #classes=[0 1], tr-prevs=[0.500, 0.500], te-prevs=[0.500, 0.500]


In [49]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

models = {}
vectorizers = {}

#Train the models for each domain
for name, dataset in datasets.items():
    print("[%s] Training the model" % name)
    vectorizers[name] = TfidfVectorizer(min_df=3, sublinear_tf=True)
    training_documents = vectorizers[name].fit_transform(dataset.training.instances)
    dataset.training = LabelledCollection(training_documents, dataset.training.labels, dataset.classes_)
    X_train, y_train = dataset.training.Xy
    model = LogisticRegression()
    model.fit(X_train, y_train)
    models[name] = model

[electronics] Training the model
(6000, 9438)
[books] Training the model
(6000, 15423)
[cds] Training the model
(6000, 14109)
[home] Training the model
(6000, 7909)


In [9]:
import pandas as pd
from IPython.display import display, HTML

df = pd.DataFrame(columns=datasets.keys(), index=models.keys())  

for model_name, model in models.items():
    for ds_name, dataset in datasets.items():
        test_documents = vectorizers[model_name].transform(dataset.test.instances)
        X_test, y_test = test_documents, dataset.test.labels
        y_pred = model.predict(X_test)
        df.loc[model_name, ds_name] = accuracy_score(y_test, y_pred)

print("Rows are the dataset in which the model was trained, columns are the dataset in which the model each model is tested")
display(HTML(df.to_html(float_format='{:20,.4f}'.format)))

Rows are the dataset in which the model was trained, columns are the dataset in which the model each model is tested


,electronics,books,cds,home
electronics,0.8770,0.7635,0.7650,0.8750
books,0.7695,0.8745,0.8185,0.7680
cds,0.7208,0.8270,0.8660,0.7385
home,0.8632,0.7592,0.7498,0.8908


### Table to put in the paper with info about the datasets

In [45]:
import pandas as pd

export_dir = '/home/pgonzalez/Dropbox/quant_datasetshift/'

info = pd.DataFrame(index=('books','electronics'),columns=('instances','p'))
info.loc['books','instances'] = len(domains_full['books'].instances)
info.loc['electronics','instances'] = len(domains_full['electronics'].instances)
info.loc['books','p'] = domains_full['books'].prevalence()[1]
info.loc['electronics','p'] = domains_full['electronics'].prevalence()[1]


with open(os.path.join(export_dir,'tables/datasets_info.tex'), 'w') as f:
    f.write(info.style.format({("p"): '{:.2f}'}).to_latex(hrules=True))